# Mais tópicos sobre orientação a objetos em Python

##  Nomes "privados" e mascarando nomes

Em Python, nomes iniciados pelo caractere ```_``` são considerados "privados" e devem ser referenciados com muito cuidado.
De fato, a recomendação é que eles jamais sejam acessados por outro que não o desenvolvedor/mantenedor do código onde eles foram definidos.

Esta exigência não é controlada pela linguagem!
De fato, considere o código:

In [ ]:
class Valor():
  """ Classe que armazena um simples valor """
  def __init__(self, x=None):
    self._valor = x

  def get(self):
    """ Retorna o valor armazenado """
    return self._valor

  def set(self, novo):
    """ Modifica o valor armazenado """
    self._valor = novo

A classe ```Valor```  armazena uma referência para um objeto no campo ```_valor```.

Este campo é acessível de fora da classe pelos métodos ```get``` e ```set```.

In [ ]:
a = Valor("teste")
print(a.get())
a.set("Novo Teste")
print(a.get())

teste
Novo Teste


No entanto, observe como o campo ```_valor``` pode ser acessado sem nenhum controle pelo interpretador:

In [ ]:
a = Valor("teste")
print(a._valor)
a._valor = "Novo Teste"
print(a._valor)

teste
Novo Teste


De fato, espera-se dos programadores que respeitem a convenção de evitar este tipo de acesso e se possível, ignorem que estes campos existem!

Existe no entanto um dilema com esta convenção.
Para entendê-lo, considere a seguinte situação:

Um novo programador deseja estender a classe valor  ```__str__``` que retorna a representação em string do valor armazenado.

Uma maneira de fazê-lo seria simplesmente usar herança para criar a nova classe:

In [ ]:
class ValorStr(Valor):
  def __str__(self):
    """ Retorna uma representação em string do valor armazenado """
    return str(self.get())


De fato, esta classe possui o comportamento desejado:

In [ ]:
a = ValorStr("teste")
print(a)
a.set("Novo Teste")
print(a)

teste
Novo Teste


O novo programador observa no entanto que se o objeto referenciado for *imutável*, o valor produzido por esta função pode ser armazenado para ser reusado futuramente (implementando assim um cache).

In [ ]:
class ValorStrCache(Valor):
  def __init__(self, obj):
    super().__init__(obj)
    self._cache = None

  def set(self, novo):
    Valor.set(self, novo)
    # Invalida o Cache
    self._cache = None

  def __str__(self):
    # Verifica se o cache é inválido
    if self._cache is None:
      self._cache = str(self.get())
    return self._cache


In [ ]:
a = ValorStrCache("teste")
print(a)
print(a)
print(a)
a.set("Novo Teste")
print(a)

teste
teste
teste
Novo Teste


Como visto, a solução envolve adicionar um novo campo à classe filha, ```_cache``` para guardar o resultado da chamada a ```str```.

Este nome (```_cache```), pode ser usado sem afetar a classe base pois ele não existia nela.

Considere no entanto o seguinte dilema: O que aconteceria se o programador original da classe ```Valor``` decidisse usar este nome no futuro?

Do mesmo modo, se o novo programador é até encorajado a ignorar existência dos campos iniciados por ```_```, como ele pode ter certeza de que é seguro usar o nome ```_cache```?

Para resolver este problema, Python introduz o mecanismo de mascaramento de nomes ("name mangling").

Nomes de atributos de classes iniciados por ```__``` (dois caracteres) e *não* terminados por ```__``` são modificados pelo interpretador de modo a inserir o nome da classe precedido de ```_```.

Este mascaramento é feito automaticamente em *todas* as referências ao nome dentro da classe. Deste modo, o programador não precisa se preocupar com ele.

Por exemplo, considere esta nova definição da clase ```Valor```:



In [ ]:
class Valor():
  """ Classe que armazena um simples valor """
  def __init__(self, x=None):
    self.__valor = x

  def get(self):
    """ Retorna o valor armazenado """
    return self.__valor

  def set(self, novo):
    """ Modifica o valor armazenado """
    self.__valor = novo

Observe que o campo ```__valor``` é referenciado diretamente por dentro do código. A nova classe continua funcionando normalmente:

In [ ]:
a = Valor("teste")
print(a.get())
a.set("Novo Teste")
print(a.get())

teste
Novo Teste


No entanto, o nome do campo foi modificado! Os acessos *externos* a ele agora geram um erro:

In [ ]:
a = Valor("teste")
print(a.__valor)

AttributeError: 'Valor' object has no attribute '__valor'

O nome verdadeiro do campo, quando referenciado por fora da classe, é ```_Valor__valor```.

In [ ]:
a = Valor("teste")
print(a._Valor__valor)

teste


Naturalmente, tal acesso direto não é recomendado.

De todo modo, o aspecto importante é que como toda classe tem um nome *distinto*, o programador sabe que este nome não será sobreposto em uma eventual herança desta classe.

In [ ]:
a = Valor("teste")
print(a.get())
a.__init__("Novo Teste")
print(a.get())

teste
Novo Teste
